# Simple and effective text clustering

### Input: list of documents:
 - HLV Park Hang Seo nói gì sau chiến thắng tưng bừng trước Pakistan?
 - HLV Park Hang-seo 'mất niềm tin', tiết lộ về 2 pha hỏng ăn penalty liên tiếp của Công Phượng
 - Xả súng kinh hoàng tại Điện Biên khiến 2 vợ chồng tử vong
 - Nghi án nổ súng ở Điện Biên, hai vợ chồng tử vong tại chỗ
 - Sập cầu ở Ý, 35 người thiệt mạng
 - Công Phượng đá hỏng 2 quả penalty, bố mẹ ở nhà nghĩ gì?
     
     
### Output: list of clusters:
 - Cluster 1:
     - HLV Park Hang Seo nói gì sau chiến thắng tưng bừng trước Pakistan?
     - HLV Park Hang-seo 'mất niềm tin', tiết lộ về 2 pha hỏng ăn penalty liên tiếp của Công Phượng
 - Cluster 2:
      - Xả súng kinh hoàng tại Điện Biên khiến 2 vợ chồng tử vong
      - Nghi án nổ súng ở Điện Biên, hai vợ chồng tử vong tại chỗ
 - Noises:
      - Sập cầu ở Ý, 35 người thiệt mạng
 
### Approach: connected components Clustering

<img src="Example.png">




# Load documents

In [1]:
import os
documents = []
for doc in os.listdir('documents/'):
    with open('documents/' + doc) as f:
        documents.append(f.read().decode('utf-8'))
        
print "Number of documents: ", len(documents)
for i in range(0,3):
    print documents[i] + '\n'

Number of documents:  91
Làm tình trên ghế Sweetbox: Khi sự riêng tư đi quá giới hạn
Khán giả bỏ tiền để tận hưởng không gian riêng tư, không bị làm phiền do 'chiếc hộp ngọt ngào' tạo ra giữa rạp chiếu phim. Nhưng đó thực tế vẫn là chốn công cộng.

Link xem trực tiếp U16 Việt Nam vs U16 Philippines
Dân Việt xin gửi tới quý độc giả link xem trực tiếp U16 Việt Nam vs U16 Philippines trong khuôn khổ lượt trận thứ 4 bảng A vòng bảng giải U16 Đông Nam Á 2018. Đây là trận đấu mà U16 Việt Nam cần thắng để tiếp tục nuôi hy vọng giành vé vào bán kết.

Lộ diện thông số kỹ thuật đặc biệt Galaxy Note 9
thegioitiepthi.vn Galaxy Note 9 sẽ ra mắt trong vài ngày tới. Mới đây một hộp bán lẻ thiết bị nghi là của Galaxy Note 9 tại Nga đã bị rò rỉ, lộ thông sồ kỹ thuật đặc biệt có ở chiếc điện thoại này.



# Pre-processing

In [2]:
from nltk import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from multiprocessing import Pool

stopwords = set(open('stopwords.txt').read().decode('utf-8').split('\n')[:-1])
puct_set = set([c for c in '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~'])

def generateBigram(paper):
    words = paper.split()
    if len(words) == 1:
        return ''
    bigrams = [words[i] + '_' + words[i+1] for i in range(0,len(words) - 1)]
    return ' '.join(bigrams)

def removeRedundant(text,redundantSet):
    words = text.split()
    for i in range(0,len(words)):
        if words[i].count('_') == 0 and (words[i] in redundantSet or words[i].isdigit()):
            words[i] = ''
        else:
            sub_words = words[i].split('_')
            if any(w in redundantSet or w.isdigit() for w in sub_words):
                words[i] = ''
    words = [w for w in words if w != '']
    words = ' '.join(words)
    return words

def preprocessing(text):
    text = ' '.join(word_tokenize(text))
    text = text.lower()
    text = ' '.join(text.split())
    text = text + generateBigram(text)
    text = removeRedundant(text,puct_set | stopwords)
    return text


pool = Pool(10)
clean_documents = pool.map(preprocessing,documents)
pool.terminate()

for i in range(0,5):
    print clean_documents[i] + '\n'


tình ghế sweetbox tư đi giới hạn khán giả tiền tận hưởng gian tư phiền 'chiếc hộp ngào rạp chiếu phim thực tế chốn công cộng .làm_tình ghế_sweetbox tư_đi giới_hạn hạn_khán khán_giả tận_hưởng 'chiếc_hộp rạp_chiếu chiếu_phim thực_tế chốn_công công_cộng

link trực tiếp u16 việt nam vs u16 philippines dân việt gửi quý độc giả link trực tiếp u16 việt nam vs u16 philippines khuôn khổ lượt trận bảng a vòng bảng giải u16 đông nam trận đấu u16 việt nam thắng tiếp tục nuôi hy vọng giành vé kết trực_tiếp tiếp_u16 u16_việt việt_nam nam_vs vs_u16 u16_philippines philippines_dân dân_việt quý_độc độc_giả giả_link trực_tiếp tiếp_u16 u16_việt việt_nam nam_vs vs_u16 u16_philippines khuôn_khổ khổ_lượt lượt_trận bảng_a a_vòng vòng_bảng bảng_giải giải_u16 u16_đông đông_nam trận_đấu u16_việt việt_nam tiếp_tục tục_nuôi nuôi_hy hy_vọng vọng_giành giành_vé

lộ diện thông kỹ thuật đặc biệt galaxy note thegioitiepthi.vn galaxy note mắt hộp lẻ thiết nghi galaxy note nga rò rỉ lộ thông sồ kỹ thuật đặc biệt điện th

# Feature Extraction = LSA = TruncatedSVD(TF-IDF)

In [3]:
vectorizer = TfidfVectorizer(token_pattern = "\S+", min_df = 2)
vectors = vectorizer.fit_transform(clean_documents)

print "Tf-idf shape: " + str(vectors.shape)

svd = TruncatedSVD(n_components=100, n_iter=10, random_state=42)
svd_vectors = svd.fit_transform(vectors)

print "Document 1's Vector : "
print svd_vectors[0]

Tf-idf shape: (91, 592)
Document 1's Vector : 
[ 2.46059673e-01 -6.93897695e-02 -1.21132144e-02 -7.39961047e-03
  1.92432455e-02 -6.50538301e-03  2.06772839e-02 -7.09620156e-02
  3.69710557e-01  1.89439544e-01  3.08607753e-01  2.17397839e-01
  9.98025329e-02  1.90757263e-02 -8.00372763e-02  1.19205025e-01
 -4.66764179e-02  3.77910420e-02  1.40400260e-02 -2.61164433e-01
 -5.38600696e-02 -1.75021607e-01 -4.21276499e-02 -1.02424464e-01
  6.14512775e-02 -7.38225589e-02  4.91832425e-02  2.53021881e-02
 -1.21698365e-01 -9.95706441e-02  2.95701298e-01 -1.75182609e-01
  1.00871532e-01 -2.54352223e-01  1.61975628e-01  1.37335543e-01
 -2.91084502e-02  2.70030223e-01 -1.35844464e-01 -3.31717698e-02
 -6.12752990e-02  7.81547015e-02  1.31720563e-01  1.32475934e-02
 -4.13174037e-02 -2.07552895e-02  3.87372128e-02 -8.27967661e-02
 -5.70708451e-03 -1.08670646e-01 -3.55012434e-02 -8.94610264e-02
  9.51899156e-02  8.66551123e-02 -7.75449694e-02 -8.00960811e-03
  3.09602534e-03  3.48211488e-02 -7.5049489

/root/anaconda2/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


## Calculate document similarity for all pair of document:
 - pairwise_distances(document1,document2)
 - pairwise_distances(document1,document3)
 - ......
 - pairwise_distances(documentN-1,documentN)

In [4]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
def distance(vecs):
    vec1 = vecs[0]
    vecAll = vecs[1]
    Dis_matrix = pairwise_distances(vec1,vecAll,metric = 'cosine',n_jobs=1)
    Dis_matrix = Dis_matrix.astype(np.float16)
    return Dis_matrix

def chunks_vec(l, n):
    for i in range(0, l.shape[0], n):
        yield l[i:i + n]

vector_chunks = list(chunks_vec(svd_vectors,1000))
vector_chunks = [(i,svd_vectors) for i in vector_chunks]

pool = Pool(2)
Dis_matrix = pool.map(distance,vector_chunks)
Dis_matrix = np.vstack(Dis_matrix)
pool.terminate()

print 'cosine distance between Document 1 and Document 2 : ', Dis_matrix[0][2]
print 'cosine distance between Document 2 and Document 3 : ', Dis_matrix[2][3]

cosine distance between Document 1 and Document 2 :  0.9663
cosine distance between Document 2 and Document 3 :  0.972


# Create graph
 - DocumentN is connected with DocumentM only if cosine_distance(DocumentN, DocumentM) < THRESHOLD
 - THRESHOLD = 0.4 (Change this)

In [5]:
from copy import deepcopy

THRESHOLD = 0.5

graph = deepcopy(Dis_matrix)
graph[graph <= THRESHOLD] = 2
graph[graph != 2] = 0
graph[graph == 2] = 1
graph = graph.astype(np.int8)

# Find connected components(Clusters)

In [6]:
from scipy.sparse.csgraph import connected_components
res = connected_components(graph,directed=False)

# Extract clusters

In [7]:
from collections import OrderedDict
cluster_labels = res[1]
num_cluster = res[0]
res_cluster = OrderedDict()

for i in range(0,len(cluster_labels)):
    if cluster_labels[i] in res_cluster: res_cluster[cluster_labels[i]].append(i)
    else: res_cluster[cluster_labels[i]] = [i]

res_cluster = [res_cluster[i] for i in range(0,num_cluster)]
res_cluster = [sorted(r) for r in res_cluster if len(r) > 1]
res_cluster.sort(key=len,reverse=True)

print "Number of cluster: ", len(res_cluster)
print "Number of clustered documents: ", len([j for i in res_cluster for j in i])
print "Number of noise documents: ", len(documents) - len([j for i in res_cluster for j in i])


Number of cluster:  14
Number of clustered documents:  69
Number of noise documents:  22


# Display result

In [8]:
for i in range(0,len(res_cluster)):
    print "Cluster " + str(i)
    for idx in res_cluster[i]:
        print documents[idx].split('\n')[0]
    print '\n'

Cluster 0
CGV thôi việc nhân viên phát tán cảnh nóng của khách hàng
Phát tán hình ảnh cặp đôi 'mây mưa', nhân viên rạp chiếu phim CGV bị cho thôi việc
Sa thải nhân viên rạp chiếu phim phát tán cảnh 'nóng'
Buộc thôi việc nhân viên phát tán hình ảnh cặp đôi thân mật thái quá trong rạp CGV
Cặp đôi thân mật quá đà trong rạp CGV: Lỗi tại 'ghế tình nhân' hay ý thức người dùng?
Có nên đặt sweetbox trong rạp chiếu phim?
Chính thức buộc thôi việc nhân viên phát tán hình ảnh cặp đôi thân mật thái quá trong rạp chiếu phim CGV
Diễn biến mới vụ nhân viên CGV phát tán ảnh 'nóng' của khách
Cho thôi việc nhân viên phát tán clip cặp đôi 'mây mưa' trong rạp CGV
Cặp đôi 'mây mưa' trong rạp CGV: Đặt biển báo, cho thôi việc nhân viên phát tán


Cluster 1
Quan tâm bảo vệ, phòng chống bạo lực, xâm hại trẻ em
Mỗi năm, cả nước có khoảng 2.000 trường hợp trẻ em bị bạo lực, xâm hại
579 đối tượng bạo lực, xâm hại tình dục trẻ em bị xử lý hình sự
Bao nhiêu trẻ em bị bạo hành, xâm hại chưa được phát hiện?
Cả trăm v